In [1]:
import torch
import numpy as np
import torch
import torch.nn as nn

from dialectic import DialecticImitation
from trainer.online_trainer import OnlineDialecticImitationTrainer
from tensordict.tensordict import TensorDict
from envs import make_env
import importlib
from common.seed import set_seed

from omegaconf import OmegaConf
cfg = OmegaConf.load('dialectic_config.yaml')
set_seed(cfg.seed)


/home/calm02/miniconda3/envs/tdmpc2/lib/python3.9/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


In [2]:
agent = DialecticImitation(cfg)
env = make_env(cfg)

In [3]:
print(agent.model)

DualModel(
  (_brain_l): Sequential(
    (0): NormedLinear(in_features=18, out_features=512, bias=True, act=Mish)
    (1): NormedLinear(in_features=512, out_features=512, bias=True, act=Mish)
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
  (_brain_r): Sequential(
    (0): NormedLinear(in_features=18, out_features=512, bias=True, act=Mish)
    (1): NormedLinear(in_features=512, out_features=512, bias=True, act=Mish)
    (2): Linear(in_features=512, out_features=6, bias=True)
  )
)


In [6]:
obs = env.reset()
tds_l = []
tds_r = []
# to_td = OnlineDialecticImitationTrainer.to_td

# obs_l = obs[agent.obs_filter_l]
# obs_r = obs[agent.obs_filter_r]

In [10]:
def to_td(obs, action=None, reward=None, mu=None, sigma_sq=None):
    """Creates a TensorDict for a new episode."""
    if isinstance(obs, dict):
        obs = TensorDict(obs, batch_size=(), device='cpu')
    else:
        obs = obs.unsqueeze(0).cpu()
    if action is None:
        action = torch.full_like(env.rand_act(), float('nan'))
    if reward is None:
        reward = torch.tensor(float('nan'))
    if mu is None:
        mu = torch.full_like(env.rand_act(), float('nan'))
    if sigma_sq is None:
        sigma_sq = torch.full_like(env.rand_act(), float('nan'))
    td = TensorDict(dict(
        obs=obs,
        action=action.unsqueeze(0),
        reward=reward.unsqueeze(0),
        mu=mu.unsqueeze(0),
        sigma_sq=sigma_sq.unsqueeze(0),
    ), batch_size=(1,))
    return td

In [11]:
# interactive
action_l, _, dist_l = agent.act(obs)
action_r, _, dist_r = agent.act(obs)

action = torch.concat([action_l, action_r], dim=1).detach()
action = action[0].detach().cpu()

In [12]:
obs, reward, done, info = env.step(action)

In [13]:
tds_l.append(to_td(obs, action, reward, dist_l[0], dist_l[1]))

AttributeError: 'Tensor' object has no attribute 'env'

In [55]:
tds_l

[]

In [46]:
info

defaultdict(float, {'success': 0.0})